

 <h1>
Welcome to the Math Question Answer Verification Competition! 🚀

The goal is to fine-tune a Llama-3-8B model to predict if a given solution to a math problem is correct or not. Your model should output True if the solution is correct, and False otherwise.

This notebook is a starter guide designed to get you up and running quickly. We'll walk through a simplified training process using a small subset of the data (5,000 examples) and lightweight parameters. The main goal here is to understand the complete workflow, from loading data to generating a submission file, not to achieve a top score.

Good luck, and have fun! 🎉

In [ ]:
import os, re
# Do this only in Colab notebooks! Otherwise use pip install unsloth
import torch; v = re.match(r"[0-9.]+", torch.__version__).group(0)
xformers = "xformers==" + ("0.0.32.post2" if v == "2.8.0" else "0.0.29.post3")
!pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf "datasets>=3.4.1,<4.0.0" "huggingface_hub>=0.34.0" hf_transfer
!pip install --no-deps unsloth
!pip install transformers==4.56.2
!pip install --no-deps trl==0.22.2

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 117.2/117.2 MB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 44.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 423.1/423.1 kB 45.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 276.7/276.7 kB 8.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.5/491.5 kB 41.4 MB/s eta 0:00:00
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
unsloth-zoo 2025.11.1 requires msgspec, which is not installed.
unsloth-zoo 2025.11.1 requires tyro, which is not installed.
unsloth-zoo 2025.11.1 requires torchao>=0.13.0, but you have torchao 0.10.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

## **Step 1: Install Necessary Libraries**

First, we need to install the required Python libraries. We'll be using the unsloth library, which provides highly efficient, memory-saving training methods for large language models, making it possible to fine-tune powerful models on a single free-tier GPU. We'll also install xformers for further optimization.


In [ ]:
# %%capture
# !pip install "unsloth[colab-new] @ git+https://github.com/unslothai/unsloth.git"
# !pip install --no-deps "xformers<0.0.26" "peft<0.12.0" "accelerate<0.32.0" "bitsandbytes<0.44.0" "transformers<4.43.0"

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# save
import os

# Define the path to save the model checkpoint in Google Drive
# save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"
save_path = "/content/drive/MyDrive/final_model"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

In [ ]:
# load
# Define the path where the model checkpoint was saved in Google Drive
# save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"
save_path = "/content/drive/MyDrive/acc67"


from unsloth import FastLanguageModel
import torch

# max_seq_length = 1024  # Choose any sequence length
max_seq_length = 2048  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from the saved path
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Prepare the loaded model for faster inference
FastLanguageModel.for_inference(model)

print(f"Model and tokenizer loaded from: {save_path}")

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.1: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.96G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/235 [00:00<?, ?B/s]

Unsloth 2025.11.1 patched 32 layers with 0 QKV layers, 0 O layers and 0 MLP layers.


Model and tokenizer loaded from: /content/drive/MyDrive/acc67


In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from tqdm.auto import tqdm
import numpy as np

# =========================================================================
# (*** 关键改动 1: 新的 CoT 推理提示词 ***)
# =========================================================================
# 这个提示词要求模型先给出推理过程，然后再给最终答案。
inference_prompt = """You are a great mathematician. Your task is to verify if a solution to a given maths question is correct.
First, provide a step-by-step reasoning by checking each part of the solution.
After your reasoning, you MUST conclude with the final answer in the format:
'Final Answer: True' or 'Final Answer: False'.

Question:
{}
Solution:
{}
Output:
"""

# =========================================================================
# (*** 关键改动 2: 新的 CoT 解析函数 ***)
# =========================================================================
# 这个函数在模型的完整输出中寻找 "Final Answer:"
def parse_output(response_text):
    # 寻找 "Final Answer:" 之后的部分
    final_answer_part = response_text.split("Final Answer:")[-1]

    # 在那一部分中，小写并检查 'true'
    if 'true' in final_answer_part.lower():
        return True

    if 'false' in final_answer_part.lower():
        return False

    # 最后的备用方案：以防模型没按指示做
    output_part = response_text.split("Output:\n")[-1]
    if 'true' in output_part.lower():
        return True

    return False

# --------- 推荐的全局加速设置 (已修复) ---------
torch.backends.cuda.matmul.allow_tf32 = True
# torch.set_float32_matmul_precision("high") # (已注释掉，因为您的 PyTorch 版本低)
USE_BF16 = torch.cuda.is_bf16_supported()

# (确保 model 和 tokenizer 是您加载的 'final_best_model')
model.eval().to("cuda")
tokenizer.padding_side = "left"
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

# =========================================================================
# (*** 关键改动 3: 新的参数 ***)
# =========================================================================
MAX_INPUT = 2048       # 和训练一致
BATCH     = 16         # (保持您修复 OOM 后的安全值)
BUCKET_SZ = 128
# --- (!!! 关键 !!!) ---
MAX_NEW   = 384        # <--- 从 4 增加到 384 (给模型思考的空间)

# --------- 1) 载入并预构建 prompt、预估长度 ---------
# (重要!) 确保 "validation_data" 是我们之前创建的 5k 原始验证集
# (即 'split_data['test']' 的结果，而不是 "formatted_validation_5k")
print(f"Loading {len(validation_data)} samples from validation set and building CoT prompts...")
ds = validation_data

def build_prompt(ex):
    # (使用我们新的 CoT "inference_prompt")
    return inference_prompt.format(ex["question"], str(ex["solution"]))

prompts = [build_prompt(ex) for ex in ds]
# (同时准备好真实标签)
labels = [str(ex["is_correct"]).strip().lower() for ex in ds]
lengths = [len(tokenizer(p, add_special_tokens=True)["input_ids"]) for p in tqdm(prompts, desc="Tokenizing lengths")]

# --------- 2) 长度分桶/排序，减少 padding 浪费 ---------
print("Sorting data by length...")
order = sorted(range(len(ds)), key=lambda i: lengths[i])
buckets = [order[i:i+BUCKET_SZ] for i in range(0, len(order), BUCKET_SZ)]
print(f"Created {len(buckets)} buckets.")

# --------- 3) 批量生成 (CoT 版本) ---------
correct = 0
total = 0
amp_dtype = torch.bfloat16 if USE_BF16 else torch.float16
autocast_ctx = torch.cuda.amp.autocast(enabled=True, dtype=amp_dtype)

print("Starting CoT batch validation...")
with torch.no_grad(), autocast_ctx:
    for bucket in tqdm(buckets, total=len(buckets), mininterval=0.5, desc="Validating Batches (CoT)"):
        for s in range(0, len(bucket), BATCH):
            idxs = bucket[s:s+BATCH]
            batch_prompts = [prompts[i] for i in idxs]
            batch_labels  = [labels[i] for i in idxs] # (获取真实标签)

            enc = tokenizer(
                batch_prompts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=MAX_INPUT,
            ).to("cuda", non_blocking=True)

            out = model.generate(
                **enc,
                max_new_tokens=MAX_NEW, # <--- (使用我们新的 384)
                do_sample=False,
                use_cache=True,
            )

            in_len = enc["input_ids"].shape[1]
            gen = out[:, in_len:]
            texts = tokenizer.batch_decode(gen, skip_special_tokens=True)

            # (使用我们新的 CoT "parse_output" 函数)
            for pred_text, true_label in zip(texts, batch_labels):
                pred_label_bool = parse_output(pred_text)
                true_label_bool = (true_label == "true")

                if pred_label_bool == true_label_bool:
                    correct += 1
            total += len(batch_labels)

# --------- 4) 打印最终准确率 ---------
accuracy = correct / total * 100
print(f"\n🎉 --- CoT 验证结果 --- 🎉")
print(f"✅ 准确率: {accuracy:.2f}% ({correct}/{total})")

Loading 1000 samples from validation set and building CoT prompts...


Tokenizing lengths:   0%|          | 0/1000 [00:00<?, ?it/s]

Sorting data by length...
Created 8 buckets.
Starting CoT batch validation...


/tmp/ipython-input-1913259824.py:90: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  autocast_ctx = torch.cuda.amp.autocast(enabled=True, dtype=amp_dtype)


Validating Batches (CoT):   0%|          | 0/8 [00:00<?, ?it/s]


🎉 --- CoT 验证结果 --- 🎉
✅ 准确率: 65.90% (659/1000)


## **Step 2: Load the Model and Tokenizer**

Next, we'll load the Llama-3-8B model, which is the only model permitted for this competition. We'll use Unsloth's FastLanguageModel to handle this efficiently.

A key technique we'll use is 4-bit quantization (load_in_4bit = True). Think of this as compressing the model's knowledge into a much smaller file size. This significantly reduces the amount of GPU memory required, allowing us to fine-tune this large model even on a free platform like Google Colab.



In [ ]:
from unsloth import FastLanguageModel
import torch

# max_seq_length = 1024  # Choose any sequence length
max_seq_length = 2048  # Choose any sequence length
dtype = None  # This will auto-detect the best data type for your GPU
load_in_4bit = True  # Use 4-bit quantization to save memory

# Load the model and tokenizer from Hugging Face
# Note: We use the base model, not a 4-bit pre-quantized one,
# to ensure we start from the official weights.
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Meta-Llama-3.1-8B", # Competition-approved model
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.10.12: Fast Llama patching. Transformers: 4.56.2.
   \\   /|    NVIDIA A100-SXM4-40GB. Num GPUs = 1. Max memory: 39.557 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.8.0+cu126. CUDA: 8.0. CUDA Toolkit: 12.6. Triton: 3.4.0
\        /    Bfloat16 = TRUE. FA [Xformers = 0.0.32.post2. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


## **Step 3: Prepare the Dataset**

This is a crucial step where we format our data into a structure the model can learn from. The process involves three parts:

1.  **Loading**: We'll load the official competition dataset from Hugging Face.
2.  **Splitting**: The full dataset is massive. For this starter notebook, we'll create a much smaller, more manageable version to speed things up: **5,000 samples for training** and **500 for validation**.
3.  **Prompting**: We will format each data sample into a clear instructional prompt. This helps the model understand its role as a mathematician verifying a solution.



In [ ]:
# The instructional prompt template for training
training_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
{}"""

# We must add an End Of Sequence (EOS) token to tell the model when a completion is finished.
EOS_TOKEN = tokenizer.eos_token

# This function formats our data samples into the prompt template.
def formatting_prompts_func(examples):
    questions = examples["question"]
    solutions = examples["solution"]
    outputs = examples["is_correct"]
    texts = []
    for question, solution, output in zip(questions, solutions, outputs):
        # Format the prompt and add the EOS token
        text = training_prompt.format(question, str(solution), str(output)) + EOS_TOKEN
        texts.append(text)
    return { "text" : texts }

In [ ]:
from datasets import load_dataset

# Load the full training dataset
full_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# Shuffle the dataset for randomness and create our smaller splits
shuffled_dataset = full_dataset.shuffle(seed=42)
train_dataset = shuffled_dataset.select(range(20000))      # Use the first 20,000 for training
validation_dataset = shuffled_dataset.select(range(20000, 22000)) # Use the next 2000 for validation

In [ ]:
from datasets import load_dataset
import torch

# --- 1. 加载 100,000 个样本的完整训练集 ---
print("Loading 100k dataset...")
full_train_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="train")

# --- 2. (关键) 将 100,000 个样本拆分为 95% 训练集 和 5% 验证集 ---
split_data = full_train_dataset.train_test_split(test_size=0.001, seed=42)

full_train_data = split_data['train'] # 95,000 个样本
validation_data = split_data['test']  # 1,000 个样本 (我们将始终使用这个)

print(f"已创建 1k 验证集，大小: {len(validation_data)}")

# --- 3. 按您的计划，创建 5k 和 10k 训练子集 ---
# (我们从 95k 训练集中抽取)
train_5k_data = full_train_data.select(range(5000))
train_10k_data = full_train_data.select(range(5000,15000))

print(f"已创建 5k 训练集，大小: {len(train_5k_data)}")
print(f"已创建 10k 训练集，大小: {len(train_10k_data)}")

# --- 4. 格式化所有数据集 ---
# (确保您的格式化函数 formatting_prompts_func 已经定义)
print("Formatting datasets...")
formatted_train_5k = train_5k_data.map(formatting_prompts_func, batched=True)
formatted_train_10k = train_10k_data.map(formatting_prompts_func, batched=True)
formatted_validation_5k = validation_data.map(formatting_prompts_func, batched=True)
print("数据准备完毕。")

# (将验证集重命名，以便后续评估脚本使用)
validation_dataset = formatted_validation_5k

Loading 100k dataset...


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/train-00001-of-00002.parquet:   0%|          | 0.00/195M [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/3.65M [00:00<?, ?B/s]

...

已创建 1k 验证集，大小: 1000
已创建 5k 训练集，大小: 5000
已创建 10k 训练集，大小: 10000
Formatting datasets...


Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

数据准备完毕。


In [ ]:
# Apply the formatting function to our training dataset
formatted_train_dataset = train_dataset.map(formatting_prompts_func, batched=True)

In [ ]:
# Trainer 会加载 checkpoint-500 并从 Step 501 开始继续
print("正在从 Step 500 恢复训练...")
trainer.train(resume_from_checkpoint = "outputs_final_run/checkpoint-500")
print("训练已从检查点恢复并完成。")

In [ ]:
# 加载检查点
!cp -r "/content/drive/My Drive/checkpoint-500" /content/outputs_final_run/


In [ ]:
# 1. 连接 Google Drive (如果还没连接)
from google.colab import drive
drive.mount('/content/drive')

# 2. 复制检查点
# !cp -r /content/outputs_final_run/checkpoint-500 /content/drive/MyDrive
# !cp -r /content/outputs_final_run/checkpoint-400 /content/drive/MyDrive

# outputs_stage1/checkpoint-60 (准确率: 67.00%)
!cp -r /content/outputs_stage1/checkpoint-60 /content/drive/MyDrive/acc67
print("Checkpoint-500 已成功备份到 Google Drive!")

## **Step 4: Configure LoRA and Set Up the Trainer**

### **LoRA Configuration**

Instead of training the entire model (which has billions of parameters), we'll use a technique called **Lo**w-**R**ank **A**daptation (LoRA). 🎛️

Think of it like this: rather than rewriting an entire textbook, we're just adding small, efficient "sticky notes" (the LoRA adapters) to update the model's knowledge. This is much faster and requires significantly less memory. We'll use a small **rank** (`r = 8`) to keep the training process light and quick for this starter notebook.


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # A small rank for lighter training
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 64, # A common practice is to set alpha = 2 * r
    lora_dropout = 0.02,
    bias = "none",
    use_gradient_checkpointing = "unsloth",
    random_state = 42,
)

Unsloth: Already have LoRA adapters! We shall skip this step.



### **SFTTrainer Setup**

Now we'll set up the `SFTTrainer` (Supervised Fine-tuning Trainer). This is the main tool from the `trl` library that will handle the entire training loop for us. We'll give it our model, tokenizer, dataset, and a set of training instructions, such as the batch size and number of epochs.

We will train for just **one epoch** (a single pass over our 5,000-sample dataset) to keep this demonstration fast.

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# --- 1. 阶段一的训练参数 (训练 5k 数据) ---
# 5k 数据 / (batch 16 * steps 4) = 157 步/轮
training_args_stage1 = TrainingArguments(
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 8,
    warmup_steps = 50,
    learning_rate = 1e-4,          # 阶段一使用较高的学习率
    lr_scheduler_type = "cosine",
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 20,            # 5k 数据，每 20 步记录一次
    optim = "adamw_8bit",
    weight_decay = 0.01,
    seed = 42,
    output_dir = "outputs_stage1", # <--- 保存到新目录
    report_to = "none",
    num_train_epochs = 2,          # 训练 2 轮

    # --- (关键) 保存检查点 ---
    save_strategy = "steps",
    save_steps = 30,              # 每 30 步保存一次 (总共约 6 个检查点)
)

# --- 2. 初始化 SFTTrainer ---
trainer_stage1 = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_5k, # <--- 使用 5k 数据
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = training_args_stage1,
)

# --- 3. 运行训练 ---
print("--- [阶段一] 开始: 训练 5k 数据... ---")
trainer_stage1.train()
print("--- [阶段一] 训练完成。所有检查点已保存到 'outputs_stage1' ---")

In [ ]:
# Prepare the model for faster inference
FastLanguageModel.for_inference(model)

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
"""

# Select a sample from the validation set
example = validation_dataset[10] # You can change the index (e.g., to 1, 2, 50)
question = example["question"]
solution = example["solution"]

# Format the prompt with the validation data
inputs = tokenizer(
[
    inference_prompt.format(question, str(solution))
], return_tensors = "pt").to("cuda")

# Generate the model's response
outputs = model.generate(**inputs, max_new_tokens = 8, use_cache = True)
response = tokenizer.batch_decode(outputs)

# Print the results
print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution)
print("\n#### MODEL'S PREDICTION ####")
# We process the output to show only the generated text
print(response[0].split("Output:\n")[1])
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

In [ ]:
import glob
import os
import pandas as pd
from datasets import load_dataset
from tqdm.auto import tqdm
import numpy as np
from unsloth import FastLanguageModel

# =============================================================================
# (这是您之前提供的“优化验证脚本”，已打包成函数)
def run_optimized_validation(model, tokenizer, validation_dataset, inference_prompt, max_seq_length):
    print("Running optimized validation...")
    model.eval().to("cuda")
    tokenizer.padding_side = "left"
    if tokenizer.pad_token_id is None: tokenizer.pad_token = tokenizer.eos_token
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.set_float32_matmul_precision("high")
    USE_BF16 = torch.cuda.is_bf16_supported()
    MAX_INPUT = max_seq_length; MAX_NEW = 8; BATCH = 16; BUCKET_SZ = 128

    def build_prompt(ex): return inference_prompt.format(ex["question"], str(ex["solution"]))
    prompts = []; labels = []
    for ex in validation_dataset:
        prompts.append(build_prompt(ex))
        labels.append(str(ex["is_correct"]).strip().lower())
    lengths = [len(tokenizer(p, add_special_tokens=True)["input_ids"]) for p in tqdm(prompts, desc="Tokenizing lengths")]
    order = sorted(range(len(prompts)), key=lambda i: lengths[i])
    buckets = [order[i:i+BUCKET_SZ] for i in range(0, len(order), BUCKET_SZ)]
    correct = 0; total = 0
    amp_dtype = torch.bfloat16 if USE_BF16 else torch.float16
    autocast_ctx = torch.cuda.amp.autocast(enabled=True, dtype=amp_dtype)
    with torch.no_grad(), autocast_ctx:
        for bucket in tqdm(buckets, total=len(buckets), desc="Evaluating Batches"):
            for s in range(0, len(bucket), BATCH):
                idxs = bucket[s:s+BATCH]; batch_prompts = [prompts[i] for i in idxs]; batch_labels  = [labels[i] for i in idxs]
                enc = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, max_length=MAX_INPUT).to("cuda", non_blocking=True)
                out = model.generate(**enc, max_new_tokens=MAX_NEW, do_sample=False, use_cache=True)
                in_len = enc["input_ids"].shape[1]; gen = out[:, in_len:]; texts = tokenizer.batch_decode(gen, skip_special_tokens=True)
                for pred_text, true_label in zip(texts, batch_labels):
                    pred_label = "true" if "true" in pred_text.lower() else "false"
                    if pred_label == true_label: correct += 1
                total += len(batch_labels)
    accuracy = correct / total * 100
    print(f"✅ Validation Accuracy: {accuracy:.2f}% ({correct}/{total})")
    return accuracy

In [ ]:
print("--- [评估阶段一] 开始: 寻找 40k 数据的最佳模型... ---")
output_dir_stage1 = "outputs_stage1"
checkpoint_dirs = sorted(
    glob.glob(f"{output_dir_stage1}/checkpoint-*"),
    key=lambda x: int(x.split('-')[-1])
)
print(f"找到 {len(checkpoint_dirs)} 个检查点: {checkpoint_dirs}")

results = {}
for checkpoint_dir in checkpoint_dirs:
    print(f"\n--- 正在评估 {checkpoint_dir} ---")
    model_checkpoint, tokenizer_checkpoint = FastLanguageModel.from_pretrained(
        model_name = checkpoint_dir, max_seq_length = max_seq_length,
        dtype = dtype, load_in_4bit = load_in_4bit,
    )
    accuracy = run_optimized_validation(
        model_checkpoint, tokenizer_checkpoint, validation_dataset,
        inference_prompt, max_seq_length
    )
    results[checkpoint_dir] = accuracy

best_checkpoint_40k = max(results, key=results.get)
print(f"\n--- [阶段一] 评估完成! 最佳模型 (5k) 是: {best_checkpoint_40k} (准确率: {results[best_checkpoint_40k]:.2f}%) ---")


In [ ]:

# =============================================================================

# --- (新) 评估循环：遍历 [阶段一] 的所有检查点 ---
print("--- [评估阶段一] 开始: 寻找 5k 数据的最佳模型... ---")
output_dir_stage1 = "outputs_stage1"
checkpoint_dirs = sorted(
    glob.glob(f"{output_dir_stage1}/checkpoint-*"),
    key=lambda x: int(x.split('-')[-1])
)
print(f"找到 {len(checkpoint_dirs)} 个检查点: {checkpoint_dirs}")

results = {}
for checkpoint_dir in checkpoint_dirs:
    print(f"\n--- 正在评估 {checkpoint_dir} ---")
    model_checkpoint, tokenizer_checkpoint = FastLanguageModel.from_pretrained(
        model_name = checkpoint_dir, max_seq_length = max_seq_length,
        dtype = dtype, load_in_4bit = load_in_4bit,
    )
    accuracy = run_optimized_validation(
        model_checkpoint, tokenizer_checkpoint, validation_dataset,
        inference_prompt, max_seq_length
    )
    results[checkpoint_dir] = accuracy

best_checkpoint_5k = max(results, key=results.get)
print(f"\n--- [阶段一] 评估完成! 最佳模型 (5k) 是: {best_checkpoint_5k} (准确率: {results[best_checkpoint_5k]:.2f}%) ---")

# --- 保存这个最佳模型，以便阶段二加载 ---
print("正在加载并保存最佳模型到 'best_5k_model'...")
model, tokenizer = FastLanguageModel.from_pretrained(
    best_checkpoint_5k, max_seq_length = max_seq_length,
    dtype = dtype, load_in_4bit = load_in_4bit,
)
model.save_pretrained("best_5k_model")
tokenizer.save_pretrained("best_5k_model")
print("保存完毕。")

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments

# --- 1. (关键) 加载 [阶段一] 的最佳模型 ---
print("--- [阶段二] 开始: 加载 'best_5k_model'... ---")
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "best_5k_model", # <--- 加载我们刚找到的最佳模型
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
print("加载完毕。")

# --- 2. 阶段二的训练参数 (训练 10k 数据) ---

training_args_stage2 = TrainingArguments(
    per_device_train_batch_size = 16,
    gradient_accumulation_steps = 4,
    warmup_steps = 20,             # 预热步数可以少一点
    learning_rate = 1e-5,          # <--- (关键) 精调时使用更低的学习率
    lr_scheduler_type = "cosine",
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 20,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    seed = 42,
    output_dir = "outputs_stage2", # <--- 保存到新的目录
    report_to = "none",
    num_train_epochs = 1,          # <--- (关键) 精调通常只需要 1 轮

    # --- 同样保存检查点 ---
    save_strategy = "steps",
    save_steps = 20,
)

# --- 3. 初始化 SFTTrainer ---
trainer_stage2 = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_10k, # <--- 使用 10k 数据
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = training_args_stage2,
)

# --- 4. 运行训练 ---
print("--- [阶段二] 开始: 在 10k 数据上精调... ---")
trainer_stage2.train()
print("--- [阶段二] 训练完成。所有检查点已保存到 'outputs_stage2' ---")

In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from tqdm.auto import tqdm
import numpy as np
import glob
import os
from unsloth import FastLanguageModel

# =============================================================================
# 直接评估stage2
# 1. 您的优化验证脚本 (已打包成一个函数)
# =============================================================================
def run_optimized_validation(model, tokenizer, validation_dataset, inference_prompt, max_seq_length):
    """
    使用批量推理和分桶，在验证集上运行评估并返回准确率。
    """

    print("Running optimized validation...")

    # ... (前面的代码，如模型设置，保持不变) ...
    model.eval().to("cuda")
    tokenizer.padding_side = "left"
    if tokenizer.pad_token_id is None: tokenizer.pad_token = tokenizer.eos_token
    torch.backends.cuda.matmul.allow_tf32 = True
    USE_BF16 = torch.cuda.is_bf16_supported()

    # --------- 3. (*** 修复在这里 ***) ---------
    MAX_INPUT = max_seq_length
    MAX_NEW   = 8

    # (!!! 关键修复 !!!)
    # 2048 长度 x 32/64 会爆显存。我们将其大幅降低。
    BATCH     = 16 # <--- 原为 64 或 32，请调低此值
                   # 如果 16 仍然 OOM，请再次重启并尝试 8

    BUCKET_SZ = 128

    # ... (函数的其余部分，如预处理、分桶、评估循环，保持不变) ...
    # ... (粘贴您完整的“批量验证”代码的其余部分) ...

    print("Preprocessing validation dataset (building prompts and lengths)...")
    def build_prompt(ex): return inference_prompt.format(ex["question"], str(ex["solution"]))
    prompts = []; labels = []
    for ex in validation_dataset:
        prompts.append(build_prompt(ex))
        labels.append(str(ex["is_correct"]).strip().lower())
    lengths = [len(tokenizer(p, add_special_tokens=True)["input_ids"]) for p in tqdm(prompts, desc="Tokenizing lengths")]
    order = sorted(range(len(prompts)), key=lambda i: lengths[i])
    buckets = [order[i:i+BUCKET_SZ] for i in range(0, len(order), BUCKET_SZ)]
    correct = 0; total = 0
    amp_dtype = torch.bfloat16 if USE_BF16 else torch.float16
    autocast_ctx = torch.cuda.amp.autocast(enabled=True, dtype=amp_dtype)
    with torch.no_grad(), autocast_ctx:
        for bucket in tqdm(buckets, total=len(buckets), desc="Evaluating Batches"):
            for s in range(0, len(bucket), BATCH):
                idxs = bucket[s:s+BATCH]; batch_prompts = [prompts[i] for i in idxs]; batch_labels  = [labels[i] for i in idxs]
                enc = tokenizer(batch_prompts, return_tensors="pt", padding=True, truncation=True, max_length=MAX_INPUT).to("cuda", non_blocking=True)
                out = model.generate(**enc, max_new_tokens=MAX_NEW, do_sample=False, use_cache=True)
                in_len = enc["input_ids"].shape[1]; gen = out[:, in_len:]; texts = tokenizer.batch_decode(gen, skip_special_tokens=True)
                for pred_text, true_label in zip(texts, batch_labels):
                    pred_label = "true" if "true" in pred_text.lower() else "false"
                    if pred_label == true_label: correct += 1
                total += len(batch_labels)
    accuracy = correct / total * 100
    print(f"✅ Validation Accuracy: {accuracy:.2f}% ({correct}/{total})")
    return accuracy

# =============================================================================
# 2. (新) 评估循环：遍历 [阶段二] 的所有检查点
# =============================================================================

# (确保这些变量在您重启后已重新定义)
# validation_dataset = formatted_validation_5k
# inference_prompt = ...
# max_seq_length = 2048
# dtype = None
# load_in_4bit = True

print("--- [评估阶段二] 开始: 寻找 20k 数据的最终模型... ---")
output_dir_stage2 = "outputs_stage2"
checkpoint_dirs = sorted(
    glob.glob(f"{output_dir_stage2}/checkpoint-*"),
    key=lambda x: int(x.split('-')[-1])
)
print(f"找到 {len(checkpoint_dirs)} 个检查点: {checkpoint_dirs}")

results = {}
for checkpoint_dir in checkpoint_dirs:
    # (您在 image_183c9e.png 中的报错发生在此处)
    print(f"\n--- 正在评估 {checkpoint_dir} ---")
    model_checkpoint, tokenizer_checkpoint = FastLanguageModel.from_pretrained(
        model_name = checkpoint_dir, max_seq_length = max_seq_length,
        dtype = dtype, load_in_4bit = load_in_4bit,
    )

    # (现在将使用 BATCH=16 来运行，不会 OOM)
    accuracy = run_optimized_validation(
        model_checkpoint, tokenizer_checkpoint, validation_dataset,
        inference_prompt, max_seq_length
    )
    results[checkpoint_dir] = accuracy

# ... (脚本的其余部分，用于找出最佳模型并保存) ...
best_checkpoint_final = max(results, key=results.get)
print(f"\n🎉 --- 最终评估完成! 最佳模型是: {best_checkpoint_final} (准确率: {results[best_checkpoint_final]:.2f}%) --- 🎉")
print("正在加载并保存最佳模型到 'final_best_model'...")
model, tokenizer = FastLanguageModel.from_pretrained(
    best_checkpoint_final, max_seq_length = max_seq_length,
    dtype = dtype, load_in_4bit = load_in_4bit,
)
model.save_pretrained("final_best_model")
tokenizer.save_pretrained("final_best_model")
print("全部完成！您现在可以使用 'final_best_model' 来生成提交文件了。")

In [ ]:
# --- (新) 评估循环：遍历 [阶段二] 的所有检查点 ---
print("--- [评估阶段二] 开始: 寻找 10k 数据的最终模型... ---")
output_dir_stage2 = "outputs_stage2"
checkpoint_dirs = sorted(
    glob.glob(f"{output_dir_stage2}/checkpoint-*"),
    key=lambda x: int(x.split('-')[-1])
)
print(f"找到 {len(checkpoint_dirs)} 个检查点: {checkpoint_dirs}")

results = {}
for checkpoint_dir in checkpoint_dirs:
    print(f"\n--- 正在评估 {checkpoint_dir} ---")
    model_checkpoint, tokenizer_checkpoint = FastLanguageModel.from_pretrained(
        model_name = checkpoint_dir, max_seq_length = max_seq_length,
        dtype = dtype, load_in_4bit = load_in_4bit,
    )
    accuracy = run_optimized_validation(
        model_checkpoint, tokenizer_checkpoint, validation_dataset,
        inference_prompt, max_seq_length
    )
    results[checkpoint_dir] = accuracy

best_checkpoint_final = max(results, key=results.get)
print(f"\n🎉 --- 最终评估完成! 最佳模型是: {best_checkpoint_final} (准确率: {results[best_checkpoint_final]:.2f}%) --- 🎉")

# --- 保存这个最终的模型，用于提交 ---
print("正在加载并保存最佳模型到 'final_best_model'...")
model, tokenizer = FastLanguageModel.from_pretrained(
    best_checkpoint_final, max_seq_length = max_seq_length,
    dtype = dtype, load_in_4bit = load_in_4bit,
)
model.save_pretrained("final_best_model")
tokenizer.save_pretrained("final_best_model")
print("全部完成！您现在可以使用 'final_best_model' 来生成提交文件了。")

In [ ]:
from trl import SFTTrainer
from transformers import TrainingArguments
import torch

# --- 1. 定义您的训练参数 (包含您的新调参 + 保存检查点) ---
training_args = TrainingArguments(
    # --- 您新的调参 (非常好！) ---
    per_device_train_batch_size = 8,
    gradient_accumulation_steps = 4,
    warmup_steps = 50,
    learning_rate = 2e-5,
    lr_scheduler_type = "cosine",

    # --- 您原有的参数 ---
    fp16 = not torch.cuda.is_bf16_supported(),
    bf16 = torch.cuda.is_bf16_supported(),
    logging_steps = 10,
    optim = "adamw_8bit",
    weight_decay = 0.01,
    seed = 42,
    output_dir = "outputs", # <--- 检查点会保存在这里
    report_to = "none",
    num_train_epochs = 2,

    # --- (关键) 重新加入“手动评估”所需的保存步骤 ---
    save_strategy = "steps",       # <--- 告诉它按步骤保存
    save_steps = 50,              # <--- 每 50 步保存一次
    # (您也可以添加 save_total_limit = 5 来防止占满磁盘)
)

# --- 2. 初始化 SFTTrainer (与之前相同) ---
trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = formatted_train_dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    args = training_args,
)

# --- 3. 运行训练 ---
print("Starting training and saving checkpoints every 50 steps...")
trainer.train()
print("Training complete. All checkpoints are saved in 'outputs' folder.")

# --- 4. (重要) ---
# 我们*不*在这里保存 'final_model'
# 因为“最佳模型”将在下一步的评估循环中被找到！

## **Step 5: Start Training\!**

Now, we'll call the `train()` function on our `trainer` object. This will kick off the fine-tuning process. Based on our settings, this will run for one full epoch over our 5,000 examples.

Grab a coffee, as this will take a few minutes\! ☕



## **Step 6: Inference and Evaluation**

Now that our model is trained, we need to test it on our validation set. We'll use a slightly different prompt for inference—one where we leave the `Output:` section blank for the model to complete.

Let's test it on a single example from our validation set to see what it predicts.

In [ ]:
# Prepare the model for faster inference
FastLanguageModel.for_inference(model)

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
"""

# Select a sample from the validation set
example = validation_dataset[10] # You can change the index (e.g., to 1, 2, 50)
question = example["question"]
solution = example["solution"]

# Format the prompt with the validation data
inputs = tokenizer(
[
    inference_prompt.format(question, str(solution))
], return_tensors = "pt").to("cuda")

# Generate the model's response
outputs = model.generate(**inputs, max_new_tokens = 8, use_cache = True)
response = tokenizer.batch_decode(outputs)

# Print the results
print("#### QUESTION ####")
print(question)
print("\n#### SOLUTION ####")
print(solution)
print("\n#### MODEL'S PREDICTION ####")
# We process the output to show only the generated text
print(response[0].split("Output:\n")[1])
print("\n#### CORRECT ANSWER ####")
print(example["is_correct"])

#### QUESTION ####
Two cards are dealt at random from a standard deck of 52 cards.  What is the probability that the first card is a $\diamondsuit$ and the second card is an ace?

#### SOLUTION ####
Since there are 4 cards that satisfy our requirement, the probability is $4/{52 \choose 2}$.
So the answer is $\boxed{1/2626}$.

#### MODEL'S PREDICTION ####
False<|end_of_text|>

#### CORRECT ANSWER ####
False


In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from tqdm.auto import tqdm

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# --------- 推荐的全局加速设置 (已修复) ---------
torch.backends.cuda.matmul.allow_tf32 = True

# --- (!!! 关键修复 1 !!!) ---
# 您的 PyTorch 版本太旧 ( < 2.1 )，不支持此功能。
# 这是一个优化选项，不是必需的，因此我们将其注释掉。
# torch.set_float32_matmul_precision("high")
# --- (!!! 修复结束 !!!) ---

USE_BF16 = torch.cuda.is_bf16_supported()

# (确保 model 和 tokenizer 变量是您加载的 'final_best_model')
model.eval().to("cuda")
tokenizer.padding_side = "left"
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

# --------- 参数区：(*** 关键修复 2 ***) ---------
MAX_INPUT = 2048       # 和训练一致
MAX_NEW   = 4
BATCH     = 16         # <--- (修复 OOM) 原为 50，已降低
                       # 如果 16 仍然 OOM，请再次重启并尝试 8
BUCKET_SZ = 64
DATASET   = "ad6398/nyu-dl-teach-maths-comp"

# --------- 1) 载入并预构建 prompt、预估长度 ---------
print("Loading test dataset and building prompts...")
ds = load_dataset(DATASET, split="test")

def build_prompt(ex):
    return inference_prompt.format(ex["question"], str(ex["solution"]))

prompts = [build_prompt(ex) for ex in ds]
lengths = [len(tokenizer(p, add_special_tokens=True)["input_ids"]) for p in tqdm(prompts, desc="Tokenizing lengths")]

# --------- 2) 长度分桶/排序，减少 padding 浪费 ---------
print("Sorting data by length...")
order = sorted(range(len(ds)), key=lambda i: lengths[i])
buckets = [order[i:i+BUCKET_SZ] for i in range(0, len(order), BUCKET_SZ)]
print(f"Created {len(buckets)} buckets.")

preds, ids = [], []

# --------- 3) 批量生成（bf16 autocast + 左填充 + 截断） ---------
amp_dtype = torch.bfloat16 if USE_BF16 else torch.float16
autocast_ctx = torch.cuda.amp.autocast(enabled=True, dtype=amp_dtype)

print("Starting batch generation...")
with torch.no_grad(), autocast_ctx:
    for bucket in tqdm(buckets, total=len(buckets), mininterval=0.5, desc="Generating Submissions"):
        # 这个桶里再按批跑
        for s in range(0, len(bucket), BATCH):
            idxs = bucket[s:s+BATCH]
            batch_prompts = [prompts[i] for i in idxs]

            enc = tokenizer(
                batch_prompts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=MAX_INPUT,
            ).to("cuda", non_blocking=True)

            out = model.generate(
                **enc,
                max_new_tokens=MAX_NEW,
                do_sample=False,
                use_cache=True,
            )

            in_len = enc["input_ids"].shape[1]
            gen = out[:, in_len:]
            texts = tokenizer.batch_decode(gen, skip_special_tokens=True)

            preds.extend(parse_output(t) for t in texts)
            ids.extend(ds[i]["id"] if "id" in ds[i] else i for i in idxs)

# --------- 4) 保存提交 ---------
print("Saving submission file...")
pd.DataFrame({"ID": ids, "is_correct": preds}).to_csv("submission.csv", index=False)
print(f"✅ Saved full submission with {len(preds)} samples to 'submission.csv'")

Loading test dataset and building prompts...


Tokenizing lengths:   0%|          | 0/10000 [00:00<?, ?it/s]

Sorting data by length...
Created 157 buckets.
Starting batch generation...


/tmp/ipython-input-1325763812.py:60: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  autocast_ctx = torch.cuda.amp.autocast(enabled=True, dtype=amp_dtype)


Generating Submissions:   0%|          | 0/157 [00:00<?, ?it/s]

Saving submission file...
✅ Saved full submission with 10000 samples to 'submission.csv'


In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from tqdm.auto import tqdm
import numpy as np
import glob
import os
from unsloth import FastLanguageModel # 确保导入

# =============================================================================
# 1. 您的优化验证脚本 (已打包成一个函数)
# =============================================================================
def run_optimized_validation(model, tokenizer, validation_dataset, inference_prompt, max_seq_length):
    """
    使用批量推理和分桶，在验证集上运行评估并返回准确率。
    """

    print("Running optimized validation...")

    # --------- 1. 推荐的全局加速设置 ---------
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.set_float32_matmul_precision("high")
    USE_BF16 = torch.cuda.is_bf16_supported()

    # --------- 2. 模型和 Tokenizer 设置 ---------
    model.eval().to("cuda") # 确保模型在 GPU 上并处于评估模式
    tokenizer.padding_side = "left" # 批量生成的关键
    if tokenizer.pad_token_id is None:
        tokenizer.pad_token = tokenizer.eos_token

    # --------- 3. 参数区：按需调整 ---------
    MAX_INPUT = max_seq_length # 与训练时一致
    MAX_NEW   = 8
    BATCH     = 64
    BUCKET_SZ = 128

    # --------- 4. 载入并预处理所有样本 ---------
    print("Preprocessing validation dataset (building prompts and lengths)...")

    def build_prompt(ex):
        return inference_prompt.format(ex["question"], str(ex["solution"]))

    prompts = []
    labels = []
    for ex in validation_dataset:
        prompts.append(build_prompt(ex))
        labels.append(str(ex["is_correct"]).strip().lower())

    lengths = [len(tokenizer(p, add_special_tokens=True)["input_ids"]) for p in tqdm(prompts, desc="Tokenizing lengths")]

    # --------- 5. 长度分桶/排序 ---------
    order = sorted(range(len(prompts)), key=lambda i: lengths[i])
    buckets = [order[i:i+BUCKET_SZ] for i in range(0, len(order), BUCKET_SZ)]

    # --------- 6. 批量生成与评估 ---------
    correct = 0
    total = 0
    amp_dtype = torch.bfloat16 if USE_BF16 else torch.float16
    autocast_ctx = torch.cuda.amp.autocast(enabled=True, dtype=amp_dtype)

    with torch.no_grad(), autocast_ctx:
        for bucket in tqdm(buckets, total=len(buckets), desc="Evaluating Batches"):
            for s in range(0, len(bucket), BATCH):
                idxs = bucket[s:s+BATCH]
                batch_prompts = [prompts[i] for i in idxs]
                batch_labels  = [labels[i] for i in idxs]

                enc = tokenizer(
                    batch_prompts,
                    return_tensors="pt",
                    padding=True,
                    truncation=True,
                    max_length=MAX_INPUT,
                ).to("cuda", non_blocking=True)

                out = model.generate(
                    **enc,
                    max_new_tokens=MAX_NEW,
                    do_sample=False,
                    use_cache=True,
                )

                in_len = enc["input_ids"].shape[1]
                gen = out[:, in_len:]
                texts = tokenizer.batch_decode(gen, skip_special_tokens=True)

                for pred_text, true_label in zip(texts, batch_labels):
                    pred_label = "true" if "true" in pred_text.lower() else "false"
                    if pred_label == true_label:
                        correct += 1
                total += len(batch_labels)

    # --------- 7. 返回最终结果 ---------
    accuracy = correct / total * 100
    print(f"✅ Validation Accuracy: {accuracy:.2f}% ({correct}/{total})")
    return accuracy

# =============================================================================
# 2. (新) 评估循环：遍历所有检查点
# =============================================================================

# (确保这些变量在您的环境中已定义)
# validation_dataset = ...
# inference_prompt = ...
# max_seq_length = 1024 (或 2048)
# dtype = None
# load_in_4bit = True

# 找到所有保存的检查点
# (假设您的 training_args.output_dir = "outputs")
output_dir = "outputs"
checkpoint_dirs = sorted(
    glob.glob(f"{output_dir}/checkpoint-*"),
    key=lambda x: int(x.split('-')[-1])
)

# (额外) 如果您想也评估最终模型，可以取消注释下面这行
# checkpoint_dirs.append(output_dir) # 添加 "outputs" 文件夹本身 (最终模型)

print(f"Found {len(checkpoint_dirs)} checkpoints to evaluate: {checkpoint_dirs}")

results = {}

# 遍历所有检查点
for checkpoint_dir in checkpoint_dirs:
    print(f"\n--- Evaluating {checkpoint_dir} ---")

    # --- A. 加载该检查点模型 ---
    model_checkpoint, tokenizer_checkpoint = FastLanguageModel.from_pretrained(
        model_name = checkpoint_dir, # <--- 加载特定的检查点
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )

    # --- B. 运行优化验证 ---
    accuracy = run_optimized_validation(
        model_checkpoint,
        tokenizer_checkpoint,
        validation_dataset,
        inference_prompt,
        max_seq_length
    )

    results[checkpoint_dir] = accuracy

# --- 3. 找出最佳模型 ---
best_checkpoint = max(results, key=results.get)
best_accuracy = results[best_checkpoint]

print(f"\n🎉 Best Model Found! 🎉")
print(f"Directory: {best_checkpoint}")
print(f"Accuracy: {best_accuracy:.2f}%")

# --- 4. (最终) 加载并保存“真正”的最佳模型 ---
print(f"Loading {best_checkpoint} as the final model...")
model, tokenizer = FastLanguageModel.from_pretrained(
    best_checkpoint,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

print("Saving best model to 'final_best_model'...")
model.save_pretrained("final_best_model")
tokenizer.save_pretrained("final_best_model")
print("Done. You can now use 'final_best_model' for submission.")

In [ ]:
print("--- [阶段二] 开始: 加载 'best_5k_model'... ---")
model, tokenizer = FastLanguageModel.from_pretrained(
    best_checkpoint_40k, # <--- 加载我们刚找到的最佳模型
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
print("加载完毕。")

## **Step 7: Generate Submission File**

This is the final step\! We will now run our fine-tuned model on the official `test` dataset.

We will loop through each example in the test set, generate a prediction, and format the results into a CSV file with two columns: `ID` and `is_correct`, as required by the competition.


In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from tqdm.auto import tqdm

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# --------- 推荐的全局加速设置（L4 支持） ---------
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")
USE_BF16 = torch.cuda.is_bf16_supported()

model.eval().to("cuda")
tokenizer.padding_side = "left"
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

# --------- 参数区：按需调 ---------
MAX_INPUT = 2048            # 和训练一致，避免被动截断
MAX_NEW   = 4
BATCH     =  50             # L4 上可试 32~96，OOM 就降
BUCKET_SZ = 64              # 分桶大小（同桶内长度接近）
DATASET   = "ad6398/nyu-dl-teach-maths-comp"

# --------- 1) 载入并预构建 prompt、预估长度 ---------
ds = load_dataset(DATASET, split="test")

def build_prompt(ex):
    return inference_prompt.format(ex["question"], str(ex["solution"]))

prompts = [build_prompt(ex) for ex in ds]
# 只做一次“轻量分词”拿长度，避免主循环里重复开销
# （不开启padding/truncation）
lengths = [len(tokenizer(p, add_special_tokens=True)["input_ids"]) for p in prompts]

# --------- 2) 长度分桶/排序，减少 padding 浪费 ---------
order = sorted(range(len(ds)), key=lambda i: lengths[i])   # 从短到长
# 也可以按桶切分（同一桶内长度更接近，padding 更少）
buckets = [order[i:i+BUCKET_SZ] for i in range(0, len(order), BUCKET_SZ)]

preds, ids = [], []

# --------- 3) 批量生成（bf16 autocast + 左填充 + 截断） ---------
amp_dtype = torch.bfloat16 if USE_BF16 else torch.float16
autocast_ctx = torch.cuda.amp.autocast(enabled=True, dtype=amp_dtype)

with torch.no_grad(), autocast_ctx:
    for bucket in tqdm(buckets, total=len(buckets), mininterval=0.5):
        # 这个桶里再按批跑
        for s in range(0, len(bucket), BATCH):
            idxs = bucket[s:s+BATCH]
            batch_prompts = [prompts[i] for i in idxs]

            enc = tokenizer(
                batch_prompts,
                return_tensors="pt",
                padding=True,
                truncation=True,
                max_length=MAX_INPUT,
            ).to("cuda", non_blocking=True)

            out = model.generate(
                **enc,
                max_new_tokens=MAX_NEW,
                do_sample=False,         # 推理更稳更快
                use_cache=True,
            )

            in_len = enc["input_ids"].shape[1]
            gen = out[:, in_len:]
            texts = tokenizer.batch_decode(gen, skip_special_tokens=True)

            preds.extend(parse_output(t) for t in texts)
            ids.extend(ds[i]["id"] if "id" in ds[i] else i for i in idxs)

# --------- 4) 保存提交 ---------
pd.DataFrame({"ID": ids, "is_correct": preds}).to_csv("submission.csv", index=False)
print(f"Saved full submission with {len(preds)} samples to 'submission.csv'")

In [ ]:
import pandas as pd
from tqdm import tqdm

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")

# SAVE THE MODEL TO DRIVE AND RUN INFERENCE
Add code to save the model checkpoint to Google Drive, load the model from the checkpoint, and generate the final submission CSV file.

## Mount google drive

### Subtask:
Mount Google Drive to save the model checkpoint.


**Reasoning**:
Mount Google Drive to save the model checkpoint.



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Save model checkpoint

### Subtask:
Save the trained model checkpoint to the specified path in Google Drive.


**Reasoning**:
Define the save path and save the model and tokenizer to Google Drive.



In [ ]:
import os

# Define the path to save the model checkpoint in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# Create the directory if it doesn't exist
os.makedirs(save_path, exist_ok=True)

# Save the model and tokenizer
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

print(f"Model checkpoint and tokenizer saved to: {save_path}")

## Load model from checkpoint

### Subtask:
Load the model from the saved checkpoint.


**Reasoning**:
Load the model and tokenizer from the saved checkpoint path in Google Drive and prepare the model for inference.



In [ ]:
# Define the path where the model checkpoint was saved in Google Drive
save_path = "/content/drive/MyDrive/llama3_8b_math_verifier_checkpoint"

# Load the model and tokenizer from the saved path
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = save_path,
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

# Prepare the loaded model for faster inference
FastLanguageModel.for_inference(model)

print(f"Model and tokenizer loaded from: {save_path}")

In [ ]:
import torch
import pandas as pd
from datasets import load_dataset
from tqdm.auto import tqdm
import numpy as np

# --------- 1. 推荐的全局加速设置 ---------
# (确保这些设置在加载模型之前或之后已经设置好)
torch.backends.cuda.matmul.allow_tf32 = True
torch.set_float32_matmul_precision("high")
USE_BF16 = torch.cuda.is_bf16_supported()

# --------- 2. 模型和 Tokenizer 设置 ---------
model.eval().to("cuda") # 确保模型在 GPU 上并处于评估模式
tokenizer.padding_side = "left" # 批量生成的关键
if tokenizer.pad_token_id is None:
    tokenizer.pad_token = tokenizer.eos_token

# --------- 3. 参数区：按需调整 ---------
# 确保 MAX_INPUT 与训练时一致
# MAX_INPUT = 1024
MAX_INPUT = 2048
# 只需要生成 "True" or "False"
MAX_NEW   = 8
# 在 A100/L4 上可以设置得很高
BATCH     = 64
# 分桶大小，64 或 128 都不错
BUCKET_SZ = 128

# --------- 4. 载入并预处理所有样本 ---------
# (假设 validation_dataset 已经加载)
print("Preprocessing validation dataset (building prompts and lengths)...")

def build_prompt(ex):
    return inference_prompt.format(ex["question"], str(ex["solution"]))

# 一次性构建所有 prompts, lengths, 和 labels
prompts = []
labels = []
lengths = []

for ex in validation_dataset:
    prompts.append(build_prompt(ex))
    labels.append(str(ex["is_correct"]).strip().lower())

# 只分词一次来获取长度 (不填充/截断)
lengths = [len(tokenizer(p, add_special_tokens=True)["input_ids"]) for p in tqdm(prompts, desc="Tokenizing lengths")]
print("Preprocessing complete.")

# --------- 5. 长度分桶/排序，减少 padding 浪费 ---------
print("Sorting data by length...")
order = sorted(range(len(prompts)), key=lambda i: lengths[i])
buckets = [order[i:i+BUCKET_SZ] for i in range(0, len(order), BUCKET_SZ)]
print(f"Created {len(buckets)} buckets.")

# --------- 6. 批量生成与评估 ---------
correct = 0
total = 0

# 设置自动混合精度 (AMP)
amp_dtype = torch.bfloat16 if USE_BF16 else torch.float16
autocast_ctx = torch.cuda.amp.autocast(enabled=True, dtype=amp_dtype)

with torch.no_grad(), autocast_ctx:
    for bucket in tqdm(buckets, total=len(buckets), desc="Evaluating Batches"):
        # 在一个桶内，分批处理
        for s in range(0, len(bucket), BATCH):
            idxs = bucket[s:s+BATCH]

            # 获取这个批次的 prompts 和 真实标签
            batch_prompts = [prompts[i] for i in idxs]
            batch_labels  = [labels[i] for i in idxs]

            # 关键：Tokenizer 负责 padding 和 truncation
            enc = tokenizer(
                batch_prompts,
                return_tensors="pt",
                padding=True,          # 填充到批次中的最长
                truncation=True,       # 截断到 MAX_INPUT
                max_length=MAX_INPUT,
            ).to("cuda", non_blocking=True)

            # 批量生成
            out = model.generate(
                **enc,
                max_new_tokens=MAX_NEW,
                do_sample=False,
                use_cache=True,
            )

            # 解码模型新生成的部分
            in_len = enc["input_ids"].shape[1]
            gen = out[:, in_len:]
            texts = tokenizer.batch_decode(gen, skip_special_tokens=True)

            # 解析预测并与真实标签比较
            for pred_text, true_label in zip(texts, batch_labels):
                pred_label = "true" if "true" in pred_text.lower() else "false"
                if pred_label == true_label:
                    correct += 1

            total += len(batch_labels)

# --------- 7. 打印最终结果 ---------
accuracy = correct / total * 100
print(f"✅ Validation Accuracy: {accuracy:.2f}% ({correct}/{total})")


## Generate submission file

### Subtask:
Generate the submission CSV file using the loaded model.


**Reasoning**:
Generate the submission CSV file by iterating through the test dataset, generating predictions using the loaded model, and saving the results to a pandas DataFrame.



In [ ]:
import pandas as pd
from tqdm import tqdm
from datasets import load_dataset

# Load the official test set
test_dataset = load_dataset("ad6398/nyu-dl-teach-maths-comp", split="test")
predictions = []

# Create the prompt template for inference (no answer included)
inference_prompt = """You are a great mathematician and you are tasked with finding if a solution to a given maths question is correct or not. Your response should be 'True' if the solution is correct, otherwise 'False'. Below is the Question and Solution.
Question:
{}
Solution:
{}
Output:
"""

# A simple function to parse 'True' or 'False' from the model's raw output
def parse_output(response_text):
    # Find the text after "Output:"
    output_part = response_text.split("Output:\n")[-1]
    # Check if "True" is in that part, case-insensitively
    if 'true' in output_part.lower():
        return True
    return False

# Loop through the test dataset and generate a prediction for each example
for example in tqdm(test_dataset):
    question = example["question"]
    solution = example["solution"]

    # Format the prompt
    prompt = inference_prompt.format(question, str(solution))
    inputs = tokenizer([prompt], return_tensors="pt").to("cuda")

    # Generate the prediction
    outputs = model.generate(**inputs, max_new_tokens=8, use_cache=True)
    response_text = tokenizer.batch_decode(outputs)[0]

    # Parse the prediction and add it to our list
    prediction = parse_output(response_text)
    predictions.append(prediction)

# Create the submission DataFrame
submission = pd.DataFrame({
    'ID': range(len(predictions)),
    'is_correct': predictions
})

# Save the DataFrame to a CSV file
submission.to_csv('submission.csv', index=False)

print("\nSubmission file 'submission.csv' created successfully!")
print("You can now download this file and submit it to the Kaggle competition.")